In [1]:
!pip install selenium
!pip install webdriver-manager

!pip install requests
!pip install pandas

In [4]:
#  Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm, trange
import re
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [29]:
def tuplas_actores(id_pelicula):
    #accedemos a la página IMDB:
    #accedemos a la página IMDB:
    driver = webdriver.Chrome() #abrir el navegador
    driver.get("https://www.imdb.com/") # abrir pagina web
    driver.maximize_window() #maximizar ventana navegador
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click() # acepto las coockies
    sleep(2)
    #Buscamos pelicula por el id, en la barra buscador:

    busqueda = driver.find_element("css selector","#suggestion-search").send_keys(id_pelicula , Keys.ENTER)
    sleep(2)

    #quiero que haga scroll hasta donde ponga 'rparto principal'

    xpath_i='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]'
    pyhton= driver.find_element(By.XPATH, xpath_i)
    driver.execute_script('arguments[0].scrollIntoView(true)', pyhton)

    #driver.execute_script("window.scrollTo(0,1500)")
    sleep(2)

        #ACCEDEMOS A 'REPARTO PRINCIPAL >'
    driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary > div > a > h3 > svg').click()
    sleep(2)
    #except:
        #driver.execute_script("window.scrollTo(0,1000)") #si hay problemas con el scroll, probar con uno mas corto.
        #sleep(3)
        #ACCEDEMOS A 'REPARTO PRINCIPAL >'
        #driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[3]/div[1]/div/a/h3').click()
        #sleep(5)
    

    lista_actores_peli_i=[]

    #ciclo for para los 10 primeros actores: range(1,12):
    for i in range(2,3):
        
        #sleep(5)
        #accedemos a la informacion del actor/actriz:
        driver.find_element("xpath", f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()
        sleep(2)
        #seleccionamos desplegable, para acceder a biografia y premios:
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografia
        sleep(2)
        #acceso a la biografia
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[2]/a').click()
        sleep(2)
        
        # obtenemos el NOMBRE del actor
        nombre= driver.find_element("xpath",'//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[2]/hgroup/h2').text
        
        try:
            #obtenemos AÑO
            año_sin_limp=driver.find_element("xpath",'//*[@id="born"]/div/div/div/a[1]').text
            año=(re.findall('\d{4}', año_sin_limp))[0] #sacamos del texto cadena de texto con 4 digitos seguidos, con regex.
        except:
            año='NULL'
        sleep(2)

        driver.back()
        #volvemos a pagina anterior, de nuevo vamos al desplegable
        #acceso a los PREMIOS
        driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/button/span').click()#acceso a la biografiasleep(5)
        sleep(2)
        driver.find_element("xpath", '/html/body/div[4]/div[2]/div/div[2]/div/div[2]/div/div[1]/ul/li[4]/a').click()#acceso a la biografia
        sleep(2)
        #creamos ciclo for para sacar las los premios:

        try:
            premios=[]
            for i in range(1,6): #solo queremos 5 pelis
                premio_sin_limpiar=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[{i}]').text
                premio_limpio= re.sub('\n',' ', premio_sin_limpiar)
                premios.append(premio_limpio)
            sleep(6)
        except:
            premios='NULL'

        sleep(2)
        driver.back()
        sleep(2)

        #hacemos scroll hasta CONOCIDO_POR:
        driver.execute_script("window.scrollTo(0,1000)")
        #xpath_d='//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[1]/hgroup/h3/span'
        #pyhton_d= driver.find_element(By.XPATH, xpath_d)
        #driver.execute_script('arguments[0].scrollIntoView(true)', pyhton_d)
        #extraemos no mas de 4 pelis por las que sea conocido:
        sleep(6)

        conocido_por=[]
        try:
            for i in range(1,5):
                try:
                    peli_conocido=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[3]/section[1]/div[2]/div/div[2]/div[{i}]/div[2]/div[1]/a').text
                    conocido_por.append(peli_conocido)
                    sleep(6)
                except:
                    continue
        except:
            conocido_por='NULL'
        sleep(10)
     
        #volvemos arriba:
        driver.execute_script("window.scrollTo(0,0)")
        sleep(5)
        #extraemos informacion que hace.

        que_hace=[]
        try:
            try:
                for z in range(1,4):
                    peli_hace=driver.find_element("xpath",f'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li[{z}]').text
                    que_hace.append(peli_hace)
                    sleep(2)
            except:
                    peli_hace=driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul/li').text
                    que_hace.append(peli_hace)
                    sleep(2)
        except:
            que_hace='NULL'

        #sacamos tupla de cada actor
        lista_infor_actor=[nombre, año, que_hace, conocido_por,premios]
        tupla_x=tuple(lista_infor_actor)
        lista_actores_peli_i.append(tupla_x)#apendeamos a la lista de actores pelicula

        sleep(6)
        driver.back()
        sleep(3)
        #volvemos a la página principal 

    return lista_actores_peli_i #obtenemos una lista con tuplas informativas de los 10 principales actores.


In [27]:
lista_id=['tt0411951',
'tt10090058']
#'tt10023150',
#'tt0425785']

In [28]:

for id_peli in tqdm(lista_id):
    tupla_una_peli=tuplas_actores(id_peli)
    print(tupla_una_peli)


 50%|█████     | 1/2 [02:02<02:02, 122.49s/it]

[('Jon Foo', '1982', ['Reparto', 'Especialistas', 'Reparto'], ['Tekken', 'Thai Dragon', 'Soldado universal: Regeneración', 'Objetivo: La Casa Blanca'], 'NULL')]


100%|██████████| 2/2 [03:33<00:00, 106.59s/it]

[('Rizky Nazar', 'NULL', ['Reparto', 'Banda sonora', 'Reparto'], [], 'NULL')]


In [ ]:
#otro sitio para sacar el año
'//*[@id="born"]/div/div/div/a[1]' 